*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [2]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding, Flatten, Dense, SimpleRNN, LSTM

print(tf.__version__)

2.12.0-dev20221212


In [3]:
train_file_path = "data/train-data.tsv"
test_file_path = "data/valid-data.tsv"
header = ["labels", "text"]

train_df = pd.read_csv(train_file_path, sep="\t", header = None, names = header)
test_df = pd.read_csv(test_file_path, sep="\t", header = None, names = header)

train_text, test_text = train_df.text, test_df.text
train_labels = np.where(train_df.labels == "ham", 0, 1)
test_labels = np.where(test_df.labels == "ham", 0, 1)


In [4]:
def text_encoded(tokenizer):
    def inner(text, maxlen, truncating = "post", padding = "post"):
        sequences = tokenizer.texts_to_sequences(text)
        padded = pad_sequences(sequences, maxlen = maxlen, padding = padding, truncating = truncating)
        return padded
    return inner



In [21]:
# following this this preprocessing and modeling tutorial 
# https://towardsdatascience.com/nlp-detecting-spam-messages-with-tensorflow-b12195b8cf0e
# Also good resource
# https://hemantranvir.medium.com/spam-detection-using-rnn-simplernn-lstm-with-step-by-step-explanation-530367608071

vocab_size = 1000
embedding_dim = 16
oov_tok = "<OOV>"
max_length = 100

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_text)
word_index = tokenizer.word_index
text_to_sequences = text_encoded(tokenizer)

In [28]:
len(word_index)

7693

In [31]:
model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    SimpleRNN(embedding_dim, return_sequences=True),
    LSTM(embedding_dim, return_sequences=True),
    Flatten(),
    Dense(6, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 100, 16)           16000     
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 100, 16)           528       
                                                                 
 lstm_7 (LSTM)               (None, 100, 16)           2112      
                                                                 
 flatten_7 (Flatten)         (None, 1600)              0         
                                                                 
 dense_14 (Dense)            (None, 6)                 9606      
                                                                 
 dense_15 (Dense)            (None, 1)                 7         
                                                                 
Total params: 28,253
Trainable params: 28,253
Non-trai

In [32]:
num_epochs = 20
padded = text_to_sequences(train_text, maxlen=max_length)
test_padded = text_to_sequences(test_text, maxlen=max_length)
history = model.fit(padded, train_labels, epochs = num_epochs, validation_data = (test_padded, test_labels))

Epoch 1/20
131/131 [==============================] - 16s 81ms/step - loss: 0.1560 - accuracy: 0.9404 - val_loss: 0.0662 - val_accuracy: 0.9770
Epoch 2/20
131/131 [==============================] - 12s 95ms/step - loss: 0.0509 - accuracy: 0.9844 - val_loss: 0.0650 - val_accuracy: 0.9813
Epoch 3/20
131/131 [==============================] - 12s 90ms/step - loss: 0.0297 - accuracy: 0.9904 - val_loss: 0.0591 - val_accuracy: 0.9828
Epoch 4/20
131/131 [==============================] - 12s 94ms/step - loss: 0.0173 - accuracy: 0.9957 - val_loss: 0.0589 - val_accuracy: 0.9835
Epoch 5/20
131/131 [==============================] - 12s 92ms/step - loss: 0.0126 - accuracy: 0.9969 - val_loss: 0.0758 - val_accuracy: 0.9835
Epoch 6/20
131/131 [==============================] - 12s 94ms/step - loss: 0.0057 - accuracy: 0.9988 - val_loss: 0.0831 - val_accuracy: 0.9835
Epoch 7/20
131/131 [==============================] - 10s 79ms/step - loss: 0.0060 - accuracy: 0.9988 - val_loss: 0.0703 - val_accuracy:

In [ ]:
import plotly.express as px

df_1 = pd.DataFrame()
df_1['Training Accuracy'] = history.history['accuracy']
df_1['Validation Accuracy'] = history.history['val_accuracy']
df_1['Training Loss'] = history.history['loss']
df_1['Validation Loss'] = history.history['val_loss']
df_1['Epochs'] = range(1, len(df_1['Training Accuracy']) + 1)

# comparing the training and validation accuracy 
fig = px.line(df_1, x='Epochs', y=['Training Accuracy', 'Validation Accuracy'], title='Training and Validation Accuracy')
fig.show()

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(text):
    features = text_to_sequences([text], max_length)
    prediction = float(model.predict(features))
    return prediction, "ham" if prediction < 0.5 else "spam"


pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 1s 714ms/step
(0.1294403374195099, 'ham')


In [ ]:
predict_message("sale today! to stop texts call 98912460324")

1/1 [==============================] - 0s 39ms/step


(0.1294403374195099, 'ham')

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 30ms/step
You haven't passed yet. Keep trying.


# Run Once

Get the data into a local storage folder

Requirement: make a `data` folder here.

In [2]:
import requests
import os

urls = ["https://cdn.freecodecamp.org/project-data/sms/train-data.tsv", "https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv"]

for url in urls:
    filename = os.path.join("data", os.path.basename(url))
    with open(filename , 'wb') as fh:
        fh.write(requests.get(url).content)